# Assignment 2
Emma McCready\
sba23001

---

# Analysis of Ireland's Transport Data

See other codebook for project background/planning and dataset sources

In [25]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import colorcet as cc
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore') 
from scipy.stats import poisson, binom, norm, shapiro
import plotly.express as px 

from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, 
                                     StratifiedKFold, learning_curve, train_test_split)
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

# Data I'm focusing further analysis on:


These datasets were explored visually in Section 6 of Codebook 1.

---

# Section 1. Statistical Analysis

First, I'll load the OECD dataset looking at the number of passengers travelling per kilometer in millions, and then prepare it. I won't limit the data to the EU14 like I did in Codebook 1, in the interest of having a larger dataset for analysis.
I didn't look at the missing values previously as I only worked with a subset of the data, so I will look at this now also:

In [27]:
df3 = pd.read_csv("oecd_IE_v_EU_passenger_transport.csv")

df3 = df3.drop(columns = ['SUBJECT', 'FREQUENCY', 'Flag Codes'])
df3 = df3.rename(columns = {'INDICATOR':'Statistic Label', 
                            'MEASURE':'UNIT',
                            'TIME':'Year'})
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   LOCATION         1320 non-null   object 
 1   Statistic Label  1320 non-null   object 
 2   UNIT             1320 non-null   object 
 3   Year             1320 non-null   int64  
 4   Value            1270 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 51.7+ KB


In [28]:
missing_count = df3['Value'].isnull().sum()

print('There are', missing_count, 'missing values in the total_migration column. The total percentage of missing values is', 
      round((missing_count/len(df3) * 100), 2), '%')

There are 50 missing values in the total_migration column. The total percentage of missing values is 3.79 %


In [29]:
missing = pd.isnull(df3['Value'])
missing_data = df3[missing]
#missing_data

Inspection of the missing data reveals that there is no data at all for Malta, presumably because there's no rail system there, so I'll just drop Malta from the dataset.

In [30]:
df3 = df3.drop(df3[df3['LOCATION'] == 'MLT'].index)
df3

,LOCATION,Statistic Label,UNIT,Year,Value
0,AUT,PASSTRANSP,MLN_PKM,1970,6281.0
1,AUT,PASSTRANSP,MLN_PKM,1971,6506.0
2,AUT,PASSTRANSP,MLN_PKM,1972,6569.0
3,AUT,PASSTRANSP,MLN_PKM,1973,6513.0
4,AUT,PASSTRANSP,MLN_PKM,1974,6575.0
...,...,...,...,...,...
1315,SVN,PASSTRANSP,MLN_PKM,2018,656.0
1316,SVN,PASSTRANSP,MLN_PKM,2019,698.0
1317,SVN,PASSTRANSP,MLN_PKM,2020,397.0
1318,SVN,PASSTRANSP,MLN_PKM,2021,542.0


The other dataset from the OECD, related to infrastructure. This data will be wrangled in the same way as Codebook 1:

In [19]:
df4 = pd.read_csv("oecd_IE_v_EU_infra_spending.csv")
df4['Value'] = df4['Value'].apply(lambda x: int(x) // 1000000 if not pd.isna(x) else x)
df4 = df4.drop(columns = ['SUBJECT', 'FREQUENCY', 'Flag Codes'])
df4 = df4.rename(columns = {'INDICATOR':'Statistic Label', 
                            'MEASURE':'UNIT',
                            'TIME':'Year',
                           }) 
df4['UNIT'] = 'EURO in millions' 
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   LOCATION         668 non-null    object 
 1   Statistic Label  668 non-null    object 
 2   UNIT             668 non-null    object 
 3   year             668 non-null    int64  
 4   Value            641 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 26.2+ KB


In [22]:
missing = pd.isnull(df4['Value'])
missing_data = df4[missing]
#missing_data

Looking at missing values again, and I note there's one again no data for Malta, so I'll drop this country from the dataset:

In [23]:
df4 = df4.drop(df4[df4['LOCATION'] == 'MLT'].index)
df4

,LOCATION,Statistic Label,UNIT,year,Value
0,AUT,INFRAINVEST,EURO in millions,1995,521.0
1,AUT,INFRAINVEST,EURO in millions,1996,590.0
2,AUT,INFRAINVEST,EURO in millions,1997,710.0
3,AUT,INFRAINVEST,EURO in millions,1998,979.0
4,AUT,INFRAINVEST,EURO in millions,1999,1120.0
...,...,...,...,...,...
663,SVN,INFRAINVEST,EURO in millions,2017,100.0
664,SVN,INFRAINVEST,EURO in millions,2018,153.0
665,SVN,INFRAINVEST,EURO in millions,2019,208.0
666,SVN,INFRAINVEST,EURO in millions,2020,189.0
